# Практические задания 18
* Лекция 36 - Парсинг веб-страниц
* Лекция 37 - MySQL и Python


1. **Парсинг данных**
> Извлеките данные с первых 10 страниц ресурса https://www.fdic.gov/bank-failures/failed-bank-list. Сохраните их в список строк таблицы. Выведите несколько произвольных строк.

*Пример вывода:*  
```
['The Woodbury Banking Company', 'Woodbury', 'Georgia', '11297', 'United Bank', 'August 19, 2016', '10521']
['First CornerStone Bank', 'King of Prussia', 'Pennsylvania', '35312', 'First-Citizens Bank & Trust Company', 'May 6, 2016', '10520']
['Trust Company Bank', 'Memphis', 'Tennessee', '9956', 'The Bank of Fayette County', 'April 29, 2016', '10519']
['North Milwaukee State Bank', 'Milwaukee', 'Wisconsin', '20364', 'First-Citizens Bank & Trust Company', 'March 11, 2016', '10518']
['Hometown National Bank', 'Longview', 'Washington', '35156', 'Twin City Bank', 'October 2, 2015', '10517']
```

In [10]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.fdic.gov/bank-failures/failed-bank-list'
data = []
for i in range(1,11):
    params = {'page': i}
    response = requests.get(url, params = params) # 'https://www.fdic.gov/bank-failures/failed-bank-list'+f'?pg={i}'      -> https://www.fdic.gov/bank-failures/failed-bank-list?pg=3

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            row_data = [col.text.strip() for col in cols]
            data.append(row_data)
    else:
        print('Ошибка запроса к странице')

for row in data[1:6]:
    print(row)

['Allied Bank', 'Mulberry', 'Arkansas', '91', "Today's Bank", 'September 23, 2016', '10522']
['The Woodbury Banking Company', 'Woodbury', 'Georgia', '11297', 'United Bank', 'August 19, 2016', '10521']
['First CornerStone Bank', 'King of Prussia', 'Pennsylvania', '35312', 'First-Citizens Bank & Trust Company', 'May 6, 2016', '10520']
['Trust Company Bank', 'Memphis', 'Tennessee', '9956', 'The Bank of Fayette County', 'April 29, 2016', '10519']
['North Milwaukee State Bank', 'Milwaukee', 'Wisconsin', '20364', 'First-Citizens Bank & Trust Company', 'March 11, 2016', '10518']


2. **Создание таблицы БД**
     
Напишите программу, которая:
* создаёт базу данных `banks_<your_group>_<your_full_name>`
* выбирает эту базу через USE 
* создает таблицу banks со структурой, идентичной таблице с сайта предыдущей задачи (но дополнительно нужно добавить первичный ключ - id)
После этого:
* вставьте одну произвольную строку в таблицу
* выполните commit() после вставки
* выведите все строки таблицы
* удалите из таблицы все строки
* убедитесь, что таблица пуста.

*Пример вывода:*  
Строки в таблице после вставки:  
`(3, 'Example Bank', 'Example City', 'Example State', '00000', 'Example Institution', datetime.date(2023, 10, 1), 10000)`  
Строки в таблице после удаления:


In [1]:
import config_connect_to_db

In [3]:
config = config_connect_to_db.cnfg

In [7]:
import pymysql

connection = pymysql.connect(**config)
cursor = connection.cursor()

db_name = 'banks_110425'
cursor.execute(f'CREATE DATABASE IF NOT EXISTS {db_name}')
cursor.execute(f'USE {db_name}')

create_table_query = '''
CREATE TABLE IF NOT EXISTS bank (
id INT AUTO_INCREMENT PRIMARY KEY,
bank_name VARCHAR(255),
city VARCHAR(255),
state VARCHAR(255),
cert VARCHAR(10),
acquiring_institution VARCHAR(255),
closing_data DATE,
fund INT
)
'''

cursor.execute(create_table_query)

insert_query = '''
INSERT INTO bank (bank_name, city, state, cert, acquiring_institution, closing_data, fund)
VALUES (%s, %s, %s, %s, %s, %s, %s)
'''

cursor.execute(insert_query, ('Example Bank', 'Example City', 'Example State', '00000', 'Example Institution', '2023-10-01', 10000))
connection.commit()

cursor.execute('SELECT * FROM bank')
for row in cursor:
    print(row)



(3, 'Example Bank', 'Example City', 'Example State', '00000', 'Example Institution', datetime.date(2023, 10, 1), 10000)
(4, 'Example Bank', 'Example City', 'Example State', '00000', 'Example Institution', datetime.date(2023, 10, 1), 10000)
(5, 'Example Bank', 'Example City', 'Example State', '00000', 'Example Institution', datetime.date(2023, 10, 1), 10000)
(6, 'Example Bank', 'Example City', 'Example State', '00000', 'Example Institution', datetime.date(2023, 10, 1), 10000)
(7, 'Example Bank', 'Example City', 'Example State', '00000', 'Example Institution', datetime.date(2023, 10, 1), 10000)


In [8]:
cursor.execute('DELETE FROM bank')
connection.commit()

print('таблица после удаления:')
cursor.execute('SELECT * FROM bank')
for row in cursor:
    print(row)

таблица после удаления:


In [9]:
cursor.close()
connection.close()

3. **Заполнение базы данных**  
> Продолжите решение предыдущих задач. Вставьте в созданную базу данных строки, полученные парсингом сайта в первой задаче. Перед вставкой убедитесь, что данные имеют верные типы (числа, даты и т.п.)   
*Пример вывода:*   
Строки в таблице после вставки:
```
(2, 'The First State Bank', 'Barboursville', 'West Virginia', '14361', 'MVB Bank, Inc.', datetime.date(2020, 4, 3), 10536)
(3, 'Ericson State Bank', 'Ericson', 'Nebraska', '18265', 'Farmers and Merchants Bank', datetime.date(2020, 2, 14), 10535)
(4, 'City National Bank of New Jersey', 'Newark', 'New Jersey', '21111', 'Industrial Bank', datetime.date(2019, 11, 1), 10534)
(5, 'Resolute Bank', 'Maumee', 'Ohio', '58317', 'Buckeye State Bank', datetime.date(2019, 10, 25), 10533)
```

In [12]:
data[1:2]

[['Allied Bank',
  'Mulberry',
  'Arkansas',
  '91',
  "Today's Bank",
  'September 23, 2016',
  '10522']]

In [ ]:
from datetime import datetime

for bd in data[1:]:
    bd[5] = 

4. **Запрос к базе данных**  
> Напишите запрос к созданной базе данных, который позволяет вывести количество банков в каждом Штате и их суммарные фонды.

*Пример вывода:*   
Количество банков и суммарные фонды по штатам:  
```
Штат: West Virginia, Количество банков: 1, Суммарные фонды: 10536
Штат: Nebraska, Количество банков: 1, Суммарные фонды: 10535
Штат: New Jersey, Количество банков: 2, Суммарные фонды: 21057
Штат: Ohio, Количество банков: 2, Суммарные фонды: 21032
Штат: Kentucky, Количество банков: 2, Суммарные фонды: 21006
Штат: Texas, Количество банков: 3, Суммарные фонды: 31510
Штат: Illinois, Количество банков: 16, Суммарные фонды: 167855
...
```